In [2]:
# !pip install -q --upgrade nltk gensim bokeh pandas

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/postas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/postas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.manifold import TSNE

In [4]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation

## Запилим пословный машинный перевод!

In [16]:
!wget -O ukr_rus.train.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vAK0SWXUqei4zTimMvIhH3ufGPsbnC_O"
!wget -O ukr_rus.test.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1W9R2F8OeKHXruo2sicZ6FgBJUTJc8Us_"
!wget -O fairy_tale.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1sq8zSroFeg_afw-60OmY8RATdu_T1tej"

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
# !pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
# from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Authentication successfull.


In [14]:
downloaded = drive.CreateFile({'id': '1d7OXuil646jUeDS1JNhP9XWlZogv6rbu'})
downloaded.GetContentFile('cc.ru.300.vec.zip')

In [15]:
downloaded = drive.CreateFile({'id': '1yAqwqgUHtMSfGS99WLGe5unSCyIXfIxi'})
downloaded.GetContentFile('cc.uk.300.vec.zip')

!unzip cc.ru.300.vec.zip
!unzip cc.uk.300.vec.zip

Archive:  cc.ru.300.vec.zip
  inflating: cc.ru.300.vec           
Archive:  cc.uk.300.vec.zip
  inflating: cc.uk.300.vec           


Напишем простенькую реализацию модели машинного перевода.

Идея основана на статье [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087.pdf). У авторов в репозитории еще много интересного: [https://github.com/facebookresearch/MUSE](https://github.com/facebookresearch/MUSE).

А мы будем переводить с украинского на русский.

In [16]:
from gensim.models import KeyedVectors

ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")
ua_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [17]:
ru_emb['август']

array([ 0.0033, -0.0322, -0.0519, -0.0808, -0.0131, -0.0468, -0.0089,
        0.0023,  0.0817,  0.0219, -0.0041,  0.0005, -0.0114,  0.0355,
        0.0282, -0.0401,  0.1017,  0.0431, -0.1122,  0.0133,  0.052 ,
       -0.0478, -0.073 ,  0.0126,  0.0648,  0.0379, -0.0122, -0.0041,
        0.024 ,  0.0135,  0.0027,  0.0064,  0.0115, -0.1203,  0.0181,
       -0.0368, -0.0515, -0.0661, -0.0053,  0.0382, -0.0103, -0.0184,
        0.0067,  0.0152,  0.0046,  0.067 ,  0.1035,  0.085 ,  0.0744,
        0.0668,  0.0475,  0.0429, -0.0545,  0.0613, -0.0466, -0.0163,
       -0.0065, -0.0271, -0.0354, -0.0168, -0.0496,  0.1036, -0.0502,
        0.1049,  0.0574, -0.0144, -0.0651,  0.0028,  0.0014, -0.0369,
        0.0042, -0.0156,  0.007 , -0.0171, -0.1097,  0.0022, -0.0058,
       -0.0288, -0.0354,  0.0969,  0.0496,  0.0658,  0.0214,  0.0009,
       -0.0617,  0.01  , -0.0261,  0.1074,  0.1178,  0.0356,  0.0596,
        0.0118,  0.0266, -0.0037, -0.0792, -0.0205,  0.0263, -0.0313,
       -0.0053,  0.0

Посмотрим на пару серпень-август (являющихся переводом)

In [18]:
ru_emb.most_similar([ru_emb["август"]])

[('август', 0.9999998807907104),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095538258552551),
 ('ноябрь', 0.893003523349762),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [19]:
ua_emb.most_similar([ua_emb["серпень"]])

[('серпень', 1.000000238418579),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810409307479858),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804074287415),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [20]:
ru_emb.most_similar([ua_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256801068782806),
 ('deteydlya', 0.23031717538833618),
 ('пресечении', 0.22632379829883575),
 ('одностороннего', 0.22608885169029236),
 ('подход', 0.22305874526500702),
 ('иболее', 0.22003725171089172),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796569228172302)]

In [21]:
def load_word_pairs(filename):
    ua_ru_pairs = []
    ua_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            ua, ru = line.rstrip().split("\t")
            if ua not in ua_emb or ru not in ru_emb:
                continue
            ua_ru_pairs.append((ua, ru))
            ua_vectors.append(ua_emb[ua])
            ru_vectors.append(ru_emb[ru])
    return ua_ru_pairs, np.array(ua_vectors), np.array(ru_vectors)


ua_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
ua_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

### Учим маппинг из одного пространства эмбеддингов в другое

У нас есть пары слов, соответствующих друг другу, и их эмбеддинги. Найдем преобразование из одного пространства в другое, чтобы приблизить известные нам слова:

$$W^*= \arg\min_W ||WX - Y||_F, \text{где} ||*||_F - \text{норма Фробениуса}$$

Эта функция очень похожа на линейную регрессию (без биаса).

**Задание** Реализуйте её - воспользуйтесь `LinearRegression` из sklearn с `fit_intercept=False`:

In [25]:
from sklearn.linear_model import LinearRegression
mapping = LinearRegression(fit_intercept=False)

In [27]:
mapping.fit(X_train, Y_train)

LinearRegression(fit_intercept=False)

In [43]:
ua_ru_test[10]

('бійка', 'драка')

In [47]:
# Чему будет соответствовать слово Бiйка ))
ru_emb.most_similar(mapping.predict([X_test[10]]))

[('драка', 0.7209880352020264),
 ('потасовка', 0.6219049692153931),
 ('стычка', 0.6196291446685791),
 ('поножовщина', 0.59937983751297),
 ('перестрелка', 0.5962169170379639),
 ('бойня', 0.5959173440933228),
 ('перепалка', 0.5883157253265381),
 ('перебранка', 0.5847620368003845),
 ('пьянка', 0.567406952381134),
 ('схватка', 0.552302896976471)]

Проверим, куда перейдет `серпень`:

In [53]:
august = mapping.predict(ua_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.854128360748291),
 ('июнь', 0.8411200046539307),
 ('март', 0.8396992087364197),
 ('сентябрь', 0.8359866738319397),
 ('февраль', 0.8329296112060547),
 ('октябрь', 0.8311845064163208),
 ('ноябрь', 0.8278921246528625),
 ('июль', 0.8234526515007019),
 ('август', 0.8120498657226562),
 ('декабрь', 0.8039001226425171)]

Должно получиться, что в топе содержатся разные месяцы, но август не первый.
- А с дракой нормально все получилось... Видимо месяцы семантически близки все друг другу... 

Будем мерять percision top-k с k = 1, 5, 10.

**Задание** Реализуйте следующую функцию:

In [88]:
def precision_at_K(pairs, mapped_vectors, K=1):
    """
    :args:
        pairs = list of right word pairs [(ua_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    
    flags = 0
    for word, vector in zip(np.array(pairs)[:,1], mapped_vectors):
        flags += np.isin(word, np.array(ru_emb.most_similar([vector]))[:K, 0])
    
    precision_val = flags / len(pairs)
    return precision_val

In [89]:
assert precision_at_K([("серпень", "август")], august, K=5) == 0.0
assert precision_at_K([("серпень", "август")], august, K=9) == 1.0
assert precision_at_K([("серпень", "август")], august, K=10) == 1.0

In [90]:
assert precision_at_K(ua_ru_test, X_test) == 0.0
assert precision_at_K(ua_ru_test, Y_test) == 1.0

In [93]:
precision_top1 = precision_at_K(ua_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision_at_K(ua_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

In [94]:
# Интересно все-же - какой он.. precision
print(f'{precision_top1=}, {precision_top5=}')

precision_top1=0.6356589147286822, precision_top5=0.813953488372093


### Улучшаем маппинг

Можно показать, что маппинг лучше строить ортогональным:
$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

Искать его можно через SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$

$$W^*=UV^T$$

**Задание** Реализуйте эту функцию.

In [108]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    
    # Надеюсь это не чит - использовать np.linalg...
    U, E, Vt = np.linalg.svd(X_train.T @ Y_train)
    return U @ Vt

In [109]:
W = learn_transform(X_train, Y_train)

In [110]:
ru_emb.most_similar([ua_emb["серпень"] @ W])

[('апрель', 0.823790967464447),
 ('сентябрь', 0.8049710988998413),
 ('март', 0.802565336227417),
 ('июнь', 0.8021842837333679),
 ('октябрь', 0.8001735210418701),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914120554924011),
 ('июль', 0.790810763835907),
 ('август', 0.7891014814376831),
 ('декабрь', 0.7686372399330139)]

In [119]:
precision_top1 = precision_at_K(ua_ru_test, X_test @ W)
precision_top5 = precision_at_K(ua_ru_test, X_test @ W, 5) 
print(f'{precision_top1=}, {precision_top5=}')

precision_top1=0.6537467700258398, precision_top5=0.8242894056847545


In [120]:
assert precision_top1 >= 0.653
assert precision_top5 >= 0.824

In [121]:
# Формально - выиграли 1-2 процента.. Но реально - мало что поменялось для серпня ))

### Пишем переводчик

Реализуем простой пословный переводчик - для каждого слова будем искать его ближайшего соседа в общем пространстве эмбеддингов. Если слова нет в эмбеддингах - просто копируем его.

In [122]:
with open("fairy_tale.txt", "r") as f:
    ua_sentences = [line.rstrip().lower() for line in f]

In [141]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    return ' '.join([ru_emb.most_similar([ua_emb[token] @ W])[0][0] if token in ua_emb else token \
                    for token in sentence.split()])

In [142]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [143]:
translate("кіт зловив мишу")

'кот поймал мышку'

In [145]:
for sentence in ua_sentences:
    print(f'src: {sentence}\ndst: {translate(sentence)}\n')

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , че и пошла раз к однії бабы добувать огня ; вошла во избу че и говорит : " хороший день тебе , бабушку !

src: дай мені огня " .
dst: дай мне огня " .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а бабка только что вынула со печи пирожок со маком , сладкий , че и согнула , чтобы он прохолов ; а лисичка ой и підгледала , че токмо что бабка качнулась во печь , чтобы достать огня , то лисичка сейчас ухватила пирожок че и деру со хаты , че , пробежать , весь мак со его виїла , а туда мусо

src: де се ти набрала стільки риби ? "
dst: куда ой ты набрала столько рыбы ? "

src: вона каже : " наловила , вовчику - братику ! "
dst: она говорит : " наловила , вовчику – братику ! "

src: а собі на думці : " подожди , і я зроблю з тобою таку штуку , як і ти зо мною " .
dst: а себе по мнении : " подожди , и мной сделаю со тобой такую штуку , как и ты За мной " .

src: - " як же ти ловила ? "
dst: – " как то ты ловила ? "

src: - " так , вовчику , уложила хвостик в ополонку , вожу тихенько да й кажу ; ловися , рибка , мала і велика !
dst: – " так , вовчику , уложила хвостик во прорубь , вожу тихонько че и говорю ; ловися , рыбка , имела и большая !

src: коли хочеш , то і ти піди , налови собі " .
dst: когда хочешь , то и ты пойди , налови себе " .

src: він побіг да зробив так , як казала лисичка .
dst: он побежал че сделал так , как говорила лисичка .

src: а лисичка стала за деревом да й дивиться ; коли у вовчика зовсім хвостик примерз , вона тоді побігла в село да й кричить : " 

In [146]:
# Нуу... такое..
# Видали переводы и получше ))
# Самое сложное в этом задании был квест - с GoogleDrive скачать эти векторные эмбеддинги от фасттекста..
# Все эти танцы с бубном с созданием приложения, добавлением API google drive, генерация ключей и тп. )))
# Неожиданное развлечение изза того, что делал в локальном ноутбуке )

# Дополнительные материалы

## Почитать
### База:  
[On word embeddings - Part 1, Sebastian Ruder](http://ruder.io/word-embeddings-1/)  
[Deep Learning, NLP, and Representations, Christopher Olah](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/)  

### Как кластеризовать смыслы многозначных слов:  
[Making Sense of Word Embeddings (2016), Pelevina et al](http://anthology.aclweb.org/W16-1620)    

### Как оценивать эмбеддинги
[Evaluation methods for unsupervised word embeddings (2015), T. Schnabel](http://www.aclweb.org/anthology/D15-1036)  
[Intrinsic Evaluation of Word Vectors Fails to Predict Extrinsic Performance (2016), B. Chiu](https://www.aclweb.org/anthology/W/W16/W16-2501.pdf)  
[Problems With Evaluation of Word Embeddings Using Word Similarity Tasks (2016), M. Faruqui](https://arxiv.org/pdf/1605.02276.pdf)  
[Improving Reliability of Word Similarity Evaluation by Redesigning Annotation Task and Performance Measure (2016), Oded Avraham, Yoav Goldberg](https://arxiv.org/pdf/1611.03641.pdf)  
[Evaluating Word Embeddings Using a Representative Suite of Practical Tasks (2016), N. Nayak](https://cs.stanford.edu/~angeli/papers/2016-acl-veceval.pdf)  


## Посмотреть
[Word Vector Representations: word2vec, Lecture 2, cs224n](https://www.youtube.com/watch?v=ERibwqs9p38)